In [1]:
import pandas as pd
import numpy as np

In [2]:
selloutData =pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")

In [3]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0
1,A60,PRODUCT59,201402,22275.0
2,A60,PRODUCT34,201402,4463.0
3,A60,PRODUCT47,201402,0.0
4,A60,PRODUCT56,201402,23.0


In [4]:
selloutData["NEW_QTY"] = np.where(selloutData["QTY"]<0,0,selloutData["QTY"])

In [5]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0


53주차 제거(타입변환)

In [6]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

개월 수 변경

In [7]:
selloutData["WEEK"] =selloutData["YEARWEEK"]%100

In [8]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2
3,A60,PRODUCT47,201402,0.0,0.0,2
4,A60,PRODUCT56,201402,23.0,23.0,2


연수 변경 (함수로 변경하기)

In [9]:
##def yearFunction(inputNumber):
#     yearValue = inputNumber/100
#     yearValue = int(yearValue)
#     return yearValue

In [10]:
# selloutData["YEAR"] = selloutData["YEARWEEK"].apply(yearFunction)
# selloutData.head()

연수 변경2 (int형으로 변환하여 인덱스 하기)

In [11]:
selloutData["YEAR"] = (selloutData["YEARWEEK"]/100).astype(int)

In [12]:
selloutData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A60,PRODUCT4,201402,71.0,71.0,2,2014
1,A60,PRODUCT59,201402,22275.0,22275.0,2,2014
2,A60,PRODUCT34,201402,4463.0,4463.0,2,2014
3,A60,PRODUCT47,201402,0.0,0.0,2,2014
4,A60,PRODUCT56,201402,23.0,23.0,2,2014


정렬(깔끔하게 정리)

In [13]:
sortKey = ["REGIONID", "PRODUCT","YEARWEEK"]

In [14]:
sortedData = selloutData.sort_values(sortKey)

In [15]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
298,A00,PRODUCT34,201401,661.0,661.0,1,2014
1757,A00,PRODUCT34,201402,679.0,679.0,2,2014
3125,A00,PRODUCT34,201403,578.0,578.0,3,2014
205,A00,PRODUCT34,201404,532.0,532.0,4,2014
4369,A00,PRODUCT34,201405,516.0,516.0,5,2014


앞 줄 인덱스 없애기 (기존 데이터에 있는 순서를 정렬 후, 새로운 인덱스와 혼란이 없기 위해 없앤다)

In [16]:
step1 = sortedData.reset_index(drop=True)
step1.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR
0,A00,PRODUCT34,201401,661.0,661.0,1,2014
1,A00,PRODUCT34,201402,679.0,679.0,2,2014
2,A00,PRODUCT34,201403,578.0,578.0,3,2014
3,A00,PRODUCT34,201404,532.0,532.0,4,2014
4,A00,PRODUCT34,201405,516.0,516.0,5,2014


### rolling 함수 사용해 이동평균 구하기 (단, 정렬 후 사용해야 한다)

1) 기본 이동평균 함수

In [17]:
step1["MA"]=step1.NEW_QTY.rolling(window=15, center=True).mean()

In [18]:
step1.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,NaN
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,NaN
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,NaN
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,NaN
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,NaN


윗 줄에 NaN 값 없애기 (min_periods:최소로 평균 구할 수 있는 정도의 값, 을 이용하기)

In [19]:
step1["MA"] = step1.NEW_QTY.rolling(window=15,center=True,min_periods=1).mean()

In [20]:
step1.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000


result라는 이름으로 step1 파일을 현재 위치에 저장하기

In [21]:
step1.to_csv("./result.csv")

merge ( groupby 그룹화하기)

In [22]:
groupKey1 =["REGIONID", "PRODUCT","YEAR"]

In [23]:
selectedData = step1.groupby(groupKey1)[["NEW_QTY"]].mean()
selectedData.head()

NEW_QTY
REGIONID PRODUCT   YEAR            
A00      PRODUCT34 2014  275.961538
                   2015   85.698113
                   2016   36.576923
         PRODUCT58 2014    2.673077
                   2015    7.547170

데이터 합치기 단계 1 - (step1 & selectedData) -단, 병합하기 위해서는 index를 분리해주어야 한다.

In [24]:
step1.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,WEEK,YEAR,MA
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000


In [25]:
newSplitedData = selectedData.reset_index(drop=False)
newSplitedData.head()

,REGIONID,PRODUCT,YEAR,NEW_QTY
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,85.698113
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,7.547170


데이터 합치기 단계 2 - key를 기준으로 데이터 조인

In [26]:
mergeKey = ["REGIONID", "PRODUCT","YEAR"]

In [27]:
mergedData = pd.merge(step1, newSplitedData, on =mergeKey, how= "left")
mergedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY_x,WEEK,YEAR,MA,NEW_QTY_y
0,A00,PRODUCT34,201401,661.0,661.0,1,2014,520.000000,275.961538
1,A00,PRODUCT34,201402,679.0,679.0,2,2014,514.444444,275.961538
2,A00,PRODUCT34,201403,578.0,578.0,3,2014,516.800000,275.961538
3,A00,PRODUCT34,201404,532.0,532.0,4,2014,503.363636,275.961538
4,A00,PRODUCT34,201405,516.0,516.0,5,2014,490.750000,275.961538


### 그룹바이 함수 만들기

In [28]:
groupKey = ["REGIONID","PRODUCT"]

In [29]:
def groupFun(eachGroup):
    eachGroup["MA3"] = eachGroup["NEW_QTY"]

In [30]:
mergedData.groupby(groupKey).apply(groupFun)

KeyError: 'NEW_QTY'